<div align="right" style="text-align: right;"><i>Peter Norvig<br>Feb 2022</i></div>

# Winning Wordle

Years ago when I did a [notebook to solve Jotto](Jotto.ipynb), I never expected that a similar word game, [Wordle](https://www.nytimes.com/games/wordle/index.html), would become so popular. Congratulations to Josh Wardle for making this happen. I [added Wordle](Jotto.ipynb#Wordle) to my old notebook, but in this notebook, I answer two questions about Wordle (based on the pre-NYTimes version, with its [word list](wordle-small.txt) of 2,315 possible words).


# (1) If I win in two guesses, am I good or lucky?

I see people brag that they won in two guesses. Does this really attest to their acumen? Or is it analagous to [Little Jack Horner](https://en.wikipedia.org/wiki/Little_Jack_Horner), who pulled out a plum and said *What a good boy am I!*, oblivious to the fact that his prosperity had more to do with the plethora of plums than with his prowess. 

What is your chance of winning on your second guess? It depends on the reply you get from the first guess, and on how many other possible words have the same reply. For example, if your guess is `HELLO`, and the reply is `.GGGG` (a miss followed by 4 green squares), then the only possible target word is `CELLO`. Less obviously, if the reply to `HELLO` is `.YGY.` (a miss, a yellow, a green, another yellow, and another miss), then the only possible target word is `ALLEY`. So in either case, you are guaranteed to win on your second guess (assuming you can recognize the sole possible word). On the other hand, if the reply is `..GY.` then the target word might be either `ALLAY` or `LILAC`, so you'd have a 50% chance of winning on the second guess. So we  have two questions:
- Q: What first guess maximizes the number of **guaranteed wins** on the second guess?
<br>A: `BRUTE` and `CHANT` guarantee you 40 wins (out of 2,315).
- Q: What first guess maximizes the number of **expected wins** on the second guess?
<br>A: `TRACE` gives you 150 expected wins  (out of 2,315). 

The probability of winning in two guesses is about 6.48%, so  the answer is: mostly lucky.

# (2) What is a winning strategy I can memorize?

There has been some nice work on defining the [optimal Wordle strategy](https://www.poirrier.ca/notes/wordle-optimal/) for various ways of posing the problem. But the strategies are all complex tree structures with thousands of branch points; not the kind of thing that can be memorized by a typical human. [Christos Papadimitriou](https://www.engineering.columbia.edu/faculty/christos-papadimitriou) had the idea of using aradically simple strategy: always choose the same first 4 guesses (regardless of the replies), and with the last two guesses, guess any word that is consistent with the replies so far. 
- Q: What simple strategy **guarantees a win** within 6 guesses?
<br>A: Guess the four words `HANDY`, `SWIFT`, `GLOVE`, `CRUMP` first.<br>For guesses 5 and 6, guess any word consistent with the replies.

If you follow this strategy, then out of 2,315 possible target words, there are:
- 4 chances that you will win on one of the first four guesses
- 2,147 chances that there will be only one consistent word left, which you will guess on the 5th guess. 
- 158 chances that there will be two consistent words left; so you'll win on either the 5th or 6th guess.
- 6 chances that there are three consistent words left, but you can guess any one and if it is wrong, the reply will tell you which of the other two to guess on the 6th guess.

With this strategy you will always win, with an average of a little over 5 guesses. That's worse than the complex strategies that are guaranteed to win in 5 guesses and have an average of around 3.5 or 3.4, but you only need to remember four words to use this strategy.

# The Details

Here are some basics, including the word list, `words`, and the `reply_for` function.

In [1]:
from typing import *
from collections import defaultdict
import random 
import functools

cache = functools.lru_cache(None)

Word  = str # A word is a lower-case string of five different letters
Reply = str # A reply is five characters taken from 'GY.': Green, Yellow, Miss
Green, Yellow, Miss = 'GY.'

! [ -e wordle-small.txt ] || curl -s -O \
  https://raw.githubusercontent.com/norvig/pytudes/main/ipynb/wordle-small.txt
words = open('wordle-small.txt').read().upper().split() #  2,315 target words

@cache
def reply_for(guess, target) -> Reply: 
    "The five-character reply for this guess on this target in Wordle."
    # We'll start by having each reply be either Green or Miss ...
    reply = [Green if guess[i] == target[i] else Miss for i in range(5)]
    # ... then we'll change the replies that should be yellow
    counts = Counter(target[i] for i in range(5) if guess[i] != target[i])
    for i in range(5):
        if reply[i] == Miss and counts[guess[i]] > 0:
            counts[guess[i]] -= 1
            reply[i] = Yellow
    return ''.join(reply)

An example of the replies you get for the guess `HELLO`, from a few possible target words:

In [2]:
few = 'HELLO WORLD CELLO ALLEY HEAVY HEART ALLAY LILAC'.split()

{w: reply_for('HELLO', w) for w in few}

{'HELLO': 'GGGGG',
 'WORLD': '...GY',
 'CELLO': '.GGGG',
 'ALLEY': '.YGY.',
 'HEAVY': 'GG...',
 'HEART': 'GG...',
 'ALLAY': '..GY.',
 'LILAC': '..GY.'}

We say that a guess **partitions** the word list into  **bins**, where each bin is labeled by a reply and has 1 or more words. In the example above, the `'..GY.'` bin has two words: `ALLAY` and `LILAC`. We can get the bin sizes as follows:

In [3]:
Counter(reply_for('HELLO', w) for w in few)

Counter({'GGGGG': 1,
         '...GY': 1,
         '.GGGG': 1,
         '.YGY.': 1,
         'GG...': 2,
         '..GY.': 2})

Here is the function to compute `bin_sizes`, and then functions to answer our questions:

In [4]:
@cache
def bin_sizes(guess) -> List[int]: 
    """Sizes of the bins when `words` are partitioned by `guess`."""
    ctr = Counter(reply_for(guess, target) for target in words)
    return list(ctr.values())

def top(n, items, key=None) -> dict:
    """Top (best) `n` {item: key(item)} pairs, as ranked by `key`."""
    return {item: key(item) for item in sorted(items, key=key, reverse=True)[:n]}

def bot(n, items, key=None) -> dict:
    """Bottom (worst) `n` {item: key(item)} pairs, as ranked by `key`."""
    return {item: key(item) for item in sorted(items, key=key)[:n]}

def wins(guess) -> int: 
    """The number of guaranteed wins on the 2nd guess (after `guess` first)."""
    return bin_sizes(guess).count(1)

def expected_wins(guess):
    """The expected number of wins on the 2nd guess (after `guess` first).
    
    With n words in a bin, you have a 1 / n chance of guessing the right one.
    And the chance of ending up in a bin is n / 2315.
    So, the expected wins is, counter-intuitively, just number of possible bins.
    """
    return len(bin_sizes(guess))

Below we see that `BRUTE` and `CHANT` give the most guaranteed wins (bins of size 1), while `TRACE` has the most expected wins (because it breaks down the answers into the most number of bins):

In [5]:
top(10, words, wins)

{'BRUTE': 40,
 'CHANT': 40,
 'METRO': 39,
 'SPILT': 39,
 'DINER': 38,
 'HORDE': 38,
 'BARON': 37,
 'BERTH': 37,
 'BURNT': 37,
 'CIDER': 37}

In [6]:
top(10, words, expected_wins)

{'TRACE': 150,
 'CRATE': 148,
 'SLATE': 147,
 'PARSE': 146,
 'CRANE': 142,
 'STALE': 142,
 'HEART': 141,
 'LEAST': 140,
 'REACT': 140,
 'CATER': 139}

Below we see that `MUMMY` is one of the worst first guess, by both metrics:

In [7]:
bot(5, words, wins)

{'MUMMY': 5, 'QUEEN': 5, 'QUEER': 5, 'QUEUE': 6, 'KIOSK': 7}

In [8]:
bot(5, words, expected_wins)

{'JAZZY': 31, 'FUZZY': 34, 'QUEUE': 35, 'FIZZY': 37, 'MUMMY': 37}

# Winning Strategy 

Christos Papadimitriou came up with a set of 4 words (`ARISE`, `CLOMP`, `THUNK`, `BAWDY`) that allow you to win over 99% of the time if you guess them first, and then guess consistent words. But I wanted to get to 100%. His set uses the letter `A` twice; I decided to:
1. Look for a set of 4 words that have 20 distinct letters (but not any of the rarest letters, `JQXZ`).
2. Check if we can always win in six guesses with that set of words. 
3. If not, generate another set and try again.

First, generating the set of four words:

In [9]:
letters = set('ABCDEFGHIKLMNOPRSTUVWY') # Missing JQXZ

def disjoint_words(n=4, words=words, letters=letters) -> Tuple[Word, ...]:
    """Tuple of `n` words made of `letters`, with no repeated letters."""
    if n == 0:
        return ()
    for w in words:
        wletters = set(w)
        if wletters.issubset(letters) and len(wletters) == 5:
            rest = disjoint_words(n - 1, words, letters - wletters)
            if rest is not None:
                return (w, *rest)
    return None

In [10]:
guesses = disjoint_words(4)
guesses

('ABHOR', 'CLEFT', 'DUMPY', 'SWING')

To check if a set of guesses can win, partition the word list into bins based on the replies to all four words, and then for every bin, check that either:
- The bin is 1 or 2 words.
- If we guess any word in the bin (for the 5th guess), we will be left with all bins of size 1 (for the 6th guess).

In [11]:
def can_win(guesses, targets=words) -> bool:
    """Will these initial guesses always lead to a win in 2 more guesses?"""
    return all(len(bin) < 2 or all(can_win_bin(w, bin) for w in bin)
               for bin in partition(guesses, targets).values())

def can_win_bin(guess: Word, bin: List[Word]) -> bool:
    """If `guess` is the first guess, can we solve the bin by the second guess?"""
    # `bin` is partitioned into bins called `bin5` by `guess`; check each one
    return all(len(bin5) == 1 
               for bin5 in partition([guess], bin).values())

def partition(guesses, targets=words) -> Dict[Tuple[Reply, ...], List[Word]]:
    """Partition `words` into bins of {(reply, ...): [word, ...]}"""
    partition = defaultdict(list)
    for target in targets:
        replies = tuple(reply_for(guess, target) for guess in guesses)
        partition[replies].append(target)
    return partition

Here's an example of how `partition` works on the two guesses `HELLO` and `WORLD`:

In [12]:
partition(('HELLO', 'WORLD'), few)

defaultdict(list,
            {('GGGGG', '.Y.G.'): ['HELLO'],
             ('...GY', 'GGGGG'): ['WORLD'],
             ('.GGGG', '.Y.G.'): ['CELLO'],
             ('.YGY.', '...Y.'): ['ALLEY'],
             ('GG...', '.....'): ['HEAVY'],
             ('GG...', '..Y..'): ['HEART'],
             ('..GY.', '...Y.'): ['ALLAY', 'LILAC']})

Now can we win with the four guesses given by `disjoint_words`?

In [13]:
can_win(guesses)

False

Too bad. Why did it fail? I'll generate some output to say why:

In [14]:
def report(guesses):
    """Print a report on these guesses: do they win or not, and why?"""
    bins = list(partition(guesses).values())
    counts = Counter(map(len, bins))
    print(f'\n{guesses} is a {"winner" if can_win(guesses) else "loser"}')
    print(f'     bin counts: {dict(counts)}')
    print(f'     bins with more than 2 words:')
    for bin in bins:
        if len(bin) > 2:
            fails = ", ".join(w for w in bin if not can_win_bin(w, bin))
            print(f'     {bin} {"*** " + fails + " will not work!" if fails else ""}')
    return guesses

In [15]:
report(guesses)


('ABHOR', 'CLEFT', 'DUMPY', 'SWING') is a loser
     bin counts: {1: 2123, 2: 79, 3: 10, 4: 1}
     bins with more than 2 words:
     ['BELIE', 'BIBLE', 'LIBEL'] 
     ['BELLE', 'BEVEL', 'BEZEL'] *** BELLE will not work!
     ['BOBBY', 'BOOBY', 'BOOZY'] 
     ['BRAKE', 'BRAVE', 'ZEBRA'] *** ZEBRA will not work!
     ['CARVE', 'CRAVE', 'CRAZE'] 
     ['EAGLE', 'GAVEL', 'LEGAL'] 
     ['GAUGE', 'GAUZE', 'VAGUE'] *** VAGUE will not work!
     ['JAUNT', 'TAUNT', 'VAUNT'] *** JAUNT, TAUNT, VAUNT will not work!
     ['PIPER', 'RIPER', 'VIPER'] *** PIPER, RIPER, VIPER will not work!
     ['RESIN', 'RINSE', 'RISEN'] 
     ['SKATE', 'STAKE', 'STATE', 'STAVE'] *** STAKE, STATE, STAVE will not work!


('ABHOR', 'CLEFT', 'DUMPY', 'SWING')

We can see that, for example, when the remaining bin is `['BELLE', 'BEVEL', 'BEZEL']`, if we guess `BELLE` on the fifth guess, then either `BEVEL` or `BEZEL` remains possible on the 6th (because we haven't tested `V` or `Z` yet). It is true that guessing `BEVEL` fifth would give you a reply that would distinguish between `BELLE` and `BEZEL` on the sixth guess, but it only counts as a winner if you can guess *any* consistent word on the fifth and sixth guesses, without having to strategize about what words remain.

Since that one wasn't a winner, I will try again. Here I will generate 200 wordsets, shuffling the words before each one so that the resulting wordsets will be different.

In [16]:
def shuffle(items: List) -> List:
    """Randomly shuffle the list, and return it."""
    random.shuffle(items)
    return items

random.seed(42)
N = 200
%time wordsets = [disjoint_words(4, shuffle(words)) for _ in range(N)]

CPU times: user 33.8 s, sys: 89 ms, total: 33.9 s
Wall time: 33.9 s


I'll report on the first three of them:

In [17]:
for ws in wordsets[:3]:
    report(ws)


('SCARY', 'FIGHT', 'PLUMB', 'WOVEN') is a loser
     bin counts: {2: 102, 1: 2089, 4: 1, 3: 6}
     bins with more than 2 words:
     ['DEBAR', 'BREAK', 'BREAD', 'REBAR'] *** BREAK, REBAR will not work!
     ['TAUNT', 'JAUNT', 'DAUNT'] *** TAUNT, JAUNT, DAUNT will not work!
     ['BOBBY', 'BOOZY', 'BOOBY'] 
     ['RUDER', 'QUEER', 'UDDER'] 
     ['RABID', 'RABBI', 'BRIAR'] *** BRIAR will not work!
     ['STATE', 'STAKE', 'SKATE'] 
     ['CHILD', 'CHILI', 'CHILL'] *** CHILD, CHILI, CHILL will not work!

('TEACH', 'DUMPY', 'BLOWN', 'FRISK') is a loser
     bin counts: {1: 2099, 2: 86, 4: 2, 3: 12}
     bins with more than 2 words:
     ['AGREE', 'GAZER', 'EAGER', 'RARER'] *** RARER will not work!
     ['OTTER', 'OVERT', 'VOTER'] 
     ['VAGUE', 'GAUGE', 'GAUZE'] *** VAGUE will not work!
     ['EAGLE', 'GAVEL', 'VALVE'] 
     ['ROVER', 'GORGE', 'ROGER'] 
     ['ANGLE', 'NAVEL', 'ANGEL'] 
     ['STAVE', 'STATE', 'STAGE'] *** STAVE, STATE, STAGE will not work!
     ['GAUNT', 'JAUNT', 'VAUN

Now I'll find all the winners out opf the 200 candidates:

In [18]:
%time winners = [ws for ws in wordsets if can_win(ws)]

for ws in winners:
    report(ws)

CPU times: user 749 ms, sys: 8.64 ms, total: 758 ms
Wall time: 757 ms

('DUTCH', 'WOVEN', 'SPRIG', 'BALMY') is a winner
     bin counts: {1: 2117, 2: 88, 3: 6, 4: 1}
     bins with more than 2 words:
     ['FINER', 'INFER', 'INNER'] 
     ['STALE', 'SLATE', 'STEAL'] 
     ['BOBBY', 'BOOZY', 'BOOBY'] 
     ['ALOFT', 'FLOAT', 'ATOLL'] 
     ['STATE', 'STEAK', 'STAKE', 'SKATE'] 
     ['PLEAT', 'LEAPT', 'PLATE'] 
     ['GLAZE', 'LEGAL', 'ALGAE'] 

('CAVIL', 'DEBUG', 'NYMPH', 'WORST') is a winner
     bin counts: {1: 2133, 2: 80, 3: 6, 4: 1}
     bins with more than 2 words:
     ['FINER', 'INFER', 'INNER'] 
     ['ALOFT', 'FLOAT', 'ALLOT'] 
     ['STATE', 'STEAK', 'STAKE', 'SKATE'] 
     ['ORDER', 'ERODE', 'ODDER'] 
     ['RIPER', 'PRIZE', 'PIPER'] 
     ['FLIER', 'RIFLE', 'FILER'] 
     ['TATTY', 'FATTY', 'TAFFY'] 

('CRAFT', 'SWING', 'HOVEL', 'DUMPY') is a winner
     bin counts: {1: 2151, 2: 73, 3: 6}
     bins with more than 2 words:
     ['AGREE', 'GAZER', 'EAGER'] 
     ['BOBBY', 'BO

It looks like about 1% or 2% of random 4-word sets are winners.

Below are the two best 4-word sets that I've come up with (in previous runs of this notebook): best in that they only have two 3-word bins. Interestingly, they both can be seen as relating to a baseball game.

In [19]:
report(('BALMY', 'PITCH', 'SWUNG', 'DROVE',))


('BALMY', 'PITCH', 'SWUNG', 'DROVE') is a winner
     bin counts: {1: 2135, 2: 87, 3: 2}
     bins with more than 2 words:
     ['ALOFT', 'FLOAT', 'ATOLL'] 
     ['STATE', 'STAKE', 'SKATE'] 


('BALMY', 'PITCH', 'SWUNG', 'DROVE')

In [20]:
report(('HANDY', 'SWIFT', 'GLOVE', 'CRUMP'))


('HANDY', 'SWIFT', 'GLOVE', 'CRUMP') is a winner
     bin counts: {1: 2151, 2: 79, 3: 2}
     bins with more than 2 words:
     ['STATE', 'STAKE', 'SKATE'] 
     ['TATTY', 'TABBY', 'BATTY'] 


('HANDY', 'SWIFT', 'GLOVE', 'CRUMP')

We can compute the mean number of guesses for the `('HANDY', 'SWIFT', 'GLOVE', 'CRUMP')` wordset:

In [21]:
(1 + 2 + 3 + 4      # could be one of the first 4 guesses
 + (2151 - 4) * 5   # 1 word bins: win in 5 guesses
 + 79 * (5 + 6)     # 2 word bins: get one of them in 5 guesses, one in 6
 + 2  * (5 + 6 + 6) # 3 word bins: get one of them in 5 guesses, the others in 6
 ) / len(words)

5.0315334773218146